In [ ]:
from tqec import Basis, NoiseModel, compile_block_graph
from tqec.computation.block_graph import BlockGraph
from tqec.computation.cube import ZXCube

# add this block graph to the gallery directory 
block_steane = BlockGraph.from_dae_file("steane_volume12.dae")

# Fill all ports with ZXZ cubes to cap them
block_steane.fill_ports(ZXCube.from_str("ZXZ"))

In [ ]:
from pathlib import Path
from typing import Literal

import matplotlib.pyplot as plt
import numpy
import sinter

from tqec.compile.specs.library.css import CSS_BLOCK_BUILDER, CSS_SUBSTITUTION_BUILDER
from tqec.compile.specs.library.zxxz import (
    ZXXZ_BLOCK_BUILDER,
    ZXXZ_SUBSTITUTION_BUILDER,
)

# Basis and NoiseModel are already imported in cell 0
from tqec.simulation.plotting.inset import plot_observable_as_inset
from tqec.simulation.simulation import start_simulation_using_sinter

# Define the path directly instead of using __file__
EXAMPLE_FOLDER = Path("/Users/kabirdubey/Projects/dev/tqec/examples/pyzx-steane")
TQEC_FOLDER = EXAMPLE_FOLDER.parent
ASSETS_FOLDER = TQEC_FOLDER / "assets"
RESULTS_FOLDER = EXAMPLE_FOLDER / "sim-results"

# Create results directory if it doesn't exist
RESULTS_FOLDER.mkdir(exist_ok=True)

def generate_graphs(style: Literal["css", "zxxz"], observable_basis: Basis) -> None:
    # 1 Create `BlockGraph` representing the computation
    block_graph = block_steane
    zx_graph = block_graph.to_zx_graph()

    # 2. Find and choose the logical observables
    correlation_surfaces = block_graph.find_correlation_surfaces()
    # Optional: filter observables here
    # correlation_surfaces = [correlation_surfaces[0]]

    block_builder = CSS_BLOCK_BUILDER if style == "css" else ZXXZ_BLOCK_BUILDER
    substitution_builder = (
        CSS_SUBSTITUTION_BUILDER if style == "css" else ZXXZ_SUBSTITUTION_BUILDER
    )
    stats = start_simulation_using_sinter(
        block_graph,
        range(1, 4),
        list(numpy.logspace(-4, -1, 10)),
        NoiseModel.uniform_depolarizing,
        manhattan_radius=2,
        block_builder=block_builder,
        substitution_builder=substitution_builder,
        observables=correlation_surfaces,
        num_workers=20,
        max_shots=10_000_000,
        max_errors=5_000,
        decoders=["pymatching"],
        print_progress=True,
    )

    for i, stat in enumerate(stats):
        fig, ax = plt.subplots()
        sinter.plot_error_rate(
            ax=ax,
            stats=stat,
            x_func=lambda stat: stat.json_metadata["p"],
            group_func=lambda stat: stat.json_metadata["d"],
        )
        plot_observable_as_inset(ax, zx_graph, correlation_surfaces[i])
        ax.grid(axis="both")
        ax.legend()
        ax.loglog()
        ax.set_title(f"{style.upper()} Logical CNOT Error Rate")
        fig.savefig(
            RESULTS_FOLDER / f"{style}_steane_result_{observable_basis}_observable_{i}.png"
        )


def main():
    generate_graphs("css", Basis.Z)
    generate_graphs("css", Basis.X)
    generate_graphs("zxxz", Basis.Z)
    generate_graphs("zxxz", Basis.X)


if __name__ == "__main__":
    main()